In [1]:
# Torch
import torch
import torch.optim as optim
from torcheval.metrics import *

import pickle
from captum.attr import *
import random
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

np.set_printoptions(threshold=np.inf)

# Custom modules
from preprocessing_post_fastsurfer.subject import *
from preprocessing_post_fastsurfer.vis import *
from ozzy_torch_utils.split_dataset import *
from ozzy_torch_utils.subject_dataset import *
from ozzy_torch_utils.plot import *
from ozzy_torch_utils.train_nn import *
from ozzy_torch_utils.model_parameters import *
from ozzy_torch_utils.init_dataloaders import *
from explain_pointnet import *

In [2]:
# Load dataset
data_path = "/uolstore/home/users/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/scratch-disk/full-datasets/hcampus-1.5T-cohort"

subject_list = find_subjects_parallel(data_path)

Csv files: ['/uolstore/home/users/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/scratch-disk/full-datasets/hcampus-1.5T-cohort/idaSearch_3_19_2025.csv']


In [3]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"

Checking the significance of left and right hippocampi on models run with both

In [4]:
subject = sample(subject_list, 1)[0]

In [5]:
# Load model
pickle_pathname = "/uolstore/home/student_lnxhome01/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/runs/run_18-03-2025_18-04-04/run_18-03-2025_18-04-04_params.pkl"

with open(pickle_pathname, 'rb') as file:
    
    model_parameters = pickle.load(file)
    
model = model_parameters.model

In [6]:
lr_cloud = np.load(os.path.join(subject.path, "Left-Hippocampus_Right-Hippocampus_aligned_cropped_mesh_downsampledcloud.npy"))

lr_attributions = pointnet_ig(model, lr_cloud, device)

lr_pred_research_group, lr_pred_class, _ = get_prediction(model, lr_cloud, device)

vis_attributions(lr_attributions, subject, lr_cloud, lr_pred_research_group)

Widget(value='<iframe src="http://localhost:37721/index.html?ui=P_0x7f279dbbf1a0_0&reconnect=auto" class="pyvi…

In [7]:
# Load model
pickle_pathname = "/uolstore/home/student_lnxhome01/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/runs/run_19-03-2025_16-12-19/run_19-03-2025_16-12-19_params.pkl"

with open(pickle_pathname, 'rb') as file:
    
    model_parameters = pickle.load(file)
    
model = model_parameters.model

In [8]:
l_cloud = np.load(os.path.join(subject.path, "Left-Hippocampus_aligned_cropped_mesh_downsampledcloud.npy"))

l_pred_research_group, l_pred_class, _ = get_prediction(model, l_cloud, device)

l_attributions = pointnet_ig(model, l_cloud, device)

vis_attributions(l_attributions, subject, l_cloud, l_pred_research_group)

Widget(value='<iframe src="http://localhost:37721/index.html?ui=P_0x7f26a61c5d00_1&reconnect=auto" class="pyvi…

Interesting experiment comparing attributions from two permutations of the same cloud

In [9]:
# Load model
pickle_pathname = "/uolstore/home/student_lnxhome01/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/runs/run_18-03-2025_15-35-05/run_18-03-2025_15-35-05_params.pkl"

with open(pickle_pathname, 'rb') as file:
    
    model_parameters = pickle.load(file)
    
model = model_parameters.model

In [10]:
cloud = np.load(os.path.join(subject.path, "Left-Hippocampus_aligned_cropped_mesh_downsampledcloud.npy"))
    

In [11]:
attributions_orig = pointnet_ig(model, cloud, device)

pred_research_group_orig, _, _ = get_prediction(model, cloud, device)

shuffler = np.random.permutation(cloud.shape[0])

unshuffler = np.argsort(shuffler)

cloud_shuffled = np.array([cloud[i] for i in shuffler])

attributions_shuffled = pointnet_ig(model, cloud_shuffled, device)

pred_research_group_shuffled, _, _ = get_prediction(model, cloud, device)

cloud_unshuffled = np.array([cloud_shuffled[i] for i in unshuffler])

attributions_unshuffled = np.array([attributions_shuffled[i] for i in unshuffler])

attributions_diff = attributions_orig - attributions_unshuffled

# NB can't really visualise attributions as they will be normalsied and look large
print(np.max(np.abs(attributions_diff)))

print(np.max(np.abs(attributions_orig)))

if pred_research_group_orig != pred_research_group_shuffled:
    
    print("Research groups are different after shuffle")
    

0.05976154424048312
0.11921980889266279


In [12]:
vis_attributions(attributions_orig, subject, cloud, pred_research_group_orig)

# These two should look identical if the method was correct
vis_attributions(attributions_shuffled, subject, cloud_shuffled, pred_research_group_shuffled)

vis_attributions(attributions_unshuffled, subject, cloud_unshuffled, pred_research_group_shuffled)

Widget(value='<iframe src="http://localhost:37721/index.html?ui=P_0x7f26a623f050_2&reconnect=auto" class="pyvi…

Widget(value='<iframe src="http://localhost:37721/index.html?ui=P_0x7f26a620be60_3&reconnect=auto" class="pyvi…

Widget(value='<iframe src="http://localhost:37721/index.html?ui=P_0x7f26a4ed0860_4&reconnect=auto" class="pyvi…

Comparison between transposing the point cloud before or after passing to integrated gradients and saliency. Treating the points as features rather than the dimensions yields better results:

In [13]:
# Load model
pickle_pathname = "/uolstore/home/student_lnxhome01/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/runs/run_18-03-2025_18-04-04/run_18-03-2025_18-04-04_params.pkl"

with open(pickle_pathname, 'rb') as file:
    
    model_parameters = pickle.load(file)
    
model = model_parameters.model

In [14]:
pred_research_group, _, _ = get_prediction(model, cloud, device)

attributions = pointnet_ig(model, cloud, device)

vis_attributions(attributions, subject, cloud, pred_research_group)

Widget(value='<iframe src="http://localhost:37721/index.html?ui=P_0x7f26a62290a0_5&reconnect=auto" class="pyvi…

In [15]:

pred_research_group, _, _ = get_prediction(model, cloud, device)

attributions = pointnet_ig_deprecated(model, cloud, device)

vis_attributions(attributions, subject, cloud, pred_research_group)

Widget(value='<iframe src="http://localhost:37721/index.html?ui=P_0x7f26a4ed3710_6&reconnect=auto" class="pyvi…

In [16]:
pred_research_group, _, _ = get_prediction(model, cloud, device)

attributions = pointnet_saliency(model, cloud, device)

vis_attributions(attributions, subject, cloud, pred_research_group)

/uolstore/home/student_lnxhome01/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/.venv/lib/python3.12/site-packages/captum/_utils/gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(


Widget(value='<iframe src="http://localhost:37721/index.html?ui=P_0x7f26a4ed2360_7&reconnect=auto" class="pyvi…

In [17]:
pred_research_group, _, _ = get_prediction(model, cloud, device)

attributions = pointnet_saliency_deprecated(model, cloud, device)

vis_attributions(attributions, subject, cloud, pred_research_group)

/uolstore/home/student_lnxhome01/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/.venv/lib/python3.12/site-packages/captum/_utils/gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(


Widget(value='<iframe src="http://localhost:37721/index.html?ui=P_0x7f26a4f73bc0_8&reconnect=auto" class="pyvi…